In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_curve, auc,classification_report

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from math import sqrt


plt.rcParams['font.sans-serif'] = ['simhei']
plt.rcParams['font.serif'] = ['simhei']
plt.rcParams['axes.unicode_minus']=False # 默认是使用Unicode负号，设置正常显示字符，如正常显示负号

import seaborn as sns
sns.set_style("darkgrid",{"font.sans-serif":['simhei','Droid Sans Fallback']})
import pandas as pd


In [17]:
data=pd.read_csv('A题-全部数据/附件2/M202.csv')
data.head()

,日期,时间,生产线编号,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
0,1,0,M202,0,1,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,M202,1,0,1,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,M202,0,1,1,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,M202,1,0,2,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,4,M202,0,1,2,1,1,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data.shape

(7528712, 37)

In [19]:
X =data[[ '时间', '物料推送气缸推送状态', '物料推送气缸收回状态', '物料推送数', '物料待抓取数',
       '放置容器数', '容器上传检测数', '填装检测数', '填装定位器固定状态', '填装定位器放开状态', '物料抓取数', '填装旋转数',
       '填装下降数', '填装数', '加盖检测数', '加盖定位数', '推盖数', '加盖下降数', '加盖数', '拧盖检测数',
       '拧盖定位数', '拧盖下降数', '拧盖旋转数', '拧盖数', '合格数', '不合格数']]

In [20]:
import joblib

# load the saved model
model = joblib.load('./modellist/Q1_物料推送装置故障1001.bin')

# predict using the loaded model
model.predict(X)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
X=np.array(X)

In [22]:
import pandas as pd
import joblib

# 假设 X 已经是一个合适的输入数据格式
# X = ...

# 加载模型
models = {
    "1001": joblib.load('./modellist/Q1_物料推送装置故障1001.bin'),
    "2001": joblib.load('./modellist/Q1_物料检测装置故障2001.bin'),
    "4003": joblib.load('./modellist/Q1_填装装置填装故障4003.bin'),
    "4001": joblib.load('./modellist/Q1_填装装置检测故障4001.bin'),
    "4002": joblib.load('./modellist/Q1_填装装置定位故障4002.bin'),
    "6002": joblib.load('./modellist/Q1_拧盖装置拧盖故障6002.bin'),
    "5002": joblib.load('./modellist/Q1_加盖装置加盖故障5002.bin'),
    "5001": joblib.load('./modellist/Q1_加盖装置定位故障5001.bin'),
    "6001": joblib.load('./modellist/Q1_拧盖装置定位故障6001.bin')
}

# 对新的输入数据 X 进行预测，并收集预测结果
predictions = {}
for code, model in models.items():
    predictions[code] = model.predict(X)




In [23]:
for i,j in predictions.items():
    data[i]=j
    print(i)
    print(data[i].value_counts())

1001
1001
0    7524544
1       4168
Name: count, dtype: int64
2001
2001
0    7528704
1          8
Name: count, dtype: int64
4003
4003
0    7528712
Name: count, dtype: int64
4001
4001
0    7526819
1       1893
Name: count, dtype: int64
4002
4002
0    7522685
1       6027
Name: count, dtype: int64
6002
6002
0    7518597
1      10115
Name: count, dtype: int64
5002
5002
0    7513708
1      15004
Name: count, dtype: int64
5001
5001
0    7517104
1      11608
Name: count, dtype: int64
6001
6001
0    7525081
1       3631
Name: count, dtype: int64


In [24]:
a=data[data['1001']==1]

In [25]:
# 根据用户的明确说明，连续的时间值表示持续的故障事件，
# 我们应该相应地将这些事件分组来计算持续时间。

# 提取唯一的日期，以便分别处理每天的数据
days = a['日期'].unique()

# 初始化一个空的DataFrame，用于存储处理后的结果
processed_data = pd.DataFrame(columns=['序号', '日期', '开始时间', '持续时长/秒'])

# 对每一天的数据进行处理，计算连续事件的持续时间
for day in days:
    # 筛选出当前天的数据，并按时间排序
    day_data = a[a['日期'] == day].sort_values(by='时间')
    
    # 初始化变量，用于跟踪连续事件
    start_time = None  # 开始时间
    last_time = None   # 上一个时间点
    duration = 0       # 持续时间
    
    # 用于存储当天事件的列表
    events = []
    
    for index, row in day_data.iterrows():
        if start_time is None:
            # 开始一个新的事件
            start_time = last_time = row['时间']
            duration = 1
        elif row['时间'] == last_time + 1:
            # 继续当前的事件
            last_time = row['时间']
            duration += 1
        else:
            # 结束当前事件并开始一个新的事件
            events.append((day, start_time, duration))
            start_time = last_time = row['时间']
            duration = 1
    
    # 不要忘记添加最后一个事件
    if start_time is not None:
        events.append((day, start_time, duration))
    
    # 将事件转换为DataFrame并追加到处理后的数据中
    day_events_df = pd.DataFrame(events, columns=['日期', '开始时间', '持续时长/秒'])
    day_events_df['序号'] = range(1, len(day_events_df) + 1)
    processed_data = pd.concat([processed_data, day_events_df], ignore_index=True)
    del processed_data['序号']

processed_data

,日期,开始时间,持续时长/秒
0,4,12520,89
1,8,2877,80
2,11,2420,77
3,15,3156,93
4,26,1465,73
...,...,...,...
57,341,14256,78
58,345,11039,97
59,345,18998,89
60,352,9662,85


In [26]:

# 统计每个月的故障总次数、最长和最短的持续时长
monthly_stats = processed_data.groupby('日期').agg(
    故障总次数=('开始时间', 'count'),
    最长持续时长=('持续时长/秒', 'max'),
    最短持续时长=('持续时长/秒', 'min')
)

monthly_stats


,故障总次数,最长持续时长,最短持续时长
日期,,,
4,1,89,89
8,1,80,80
11,1,77,77
15,1,93,93
26,2,73,3
29,1,92,92
40,1,66,66
43,1,70,70
53,1,69,69


In [27]:
shapemax=0
temp=pd.DataFrame()
for d in  ['1001','2001','4001','4002','4003','5001','5002','6001','6002']:
    a=data[data[d]==1]
    # 根据用户的明确说明，连续的时间值表示持续的故障事件，
    # 我们应该相应地将这些事件分组来计算持续时间。

    # 提取唯一的日期，以便分别处理每天的数据
    days = a['日期'].unique()

    # 初始化一个空的DataFrame，用于存储处理后的结果
    processed_data = pd.DataFrame(columns=['序号', '日期', '开始时间', '持续时长/秒'])

    # 对每一天的数据进行处理，计算连续事件的持续时间
    for day in days:
        # 筛选出当前天的数据，并按时间排序
        day_data = a[a['日期'] == day].sort_values(by='时间')

        # 初始化变量，用于跟踪连续事件
        start_time = None  # 开始时间
        last_time = None   # 上一个时间点
        duration = 0       # 持续时间

        # 用于存储当天事件的列表
        events = []

        for index, row in day_data.iterrows():
            if start_time is None:
                # 开始一个新的事件
                start_time = last_time = row['时间']
                duration = 1
            elif row['时间'] == last_time + 1:
                # 继续当前的事件
                last_time = row['时间']
                duration += 1
            else:
                # 结束当前事件并开始一个新的事件
                events.append((day, start_time, duration))
                start_time = last_time = row['时间']
                duration = 1

        # 不要忘记添加最后一个事件
        if start_time is not None:
            events.append((day, start_time, duration))

        # 将事件转换为DataFrame并追加到处理后的数据中
        day_events_df = pd.DataFrame(events, columns=['日期', '开始时间', '持续时长/秒'])
        day_events_df['序号'] = range(1, len(day_events_df) + 1)
        processed_data = pd.concat([processed_data, day_events_df], ignore_index=True)
    if shapemax<processed_data.shape[0]:
        shapemax=processed_data.shape[0]
    del processed_data['序号']
    print(temp.shape)
    temp=pd.concat([temp, processed_data],axis=1)
            # 统计每个月的故障总次数、最长和最短的持续时长
    monthly_stats = processed_data.groupby('日期').agg(
        故障总次数=('开始时间', 'count'),
        最长持续时长=('持续时长/秒', 'max'),
        最短持续时长=('持续时长/秒', 'min')
    )
    print(d)
    display(monthly_stats)


(0, 0)
1001


,故障总次数,最长持续时长,最短持续时长
日期,,,
4,1,89,89
8,1,80,80
11,1,77,77
15,1,93,93
26,2,73,3
29,1,92,92
40,1,66,66
43,1,70,70
53,1,69,69


(62, 3)
2001


,故障总次数,最长持续时长,最短持续时长
日期,,,
113,1,7,7
192,1,1,1


(62, 6)
4001


...

,故障总次数,最长持续时长,最短持续时长
日期,,,
1,25,6,1
2,7,1,1
3,41,3,1
4,88,8,1
5,7,1,1
...,...,...,...
358,10,5,1
359,7,1,1
360,7,1,1


(8808, 21)
6001


,故障总次数,最长持续时长,最短持续时长
日期,,,
1,7,1,1
2,2,1,1
3,8,1,1
4,22,5,1
5,2,1,1
...,...,...,...
358,2,4,1
359,2,1,1
360,2,1,1


(8808, 24)
6002


,故障总次数,最长持续时长,最短持续时长
日期,,,
1,10,3,1
3,15,3,1
4,52,75,1
8,25,11,1
11,45,8,1
...,...,...,...
353,2,1,1
354,25,5,1
358,5,8,1


In [28]:
# 新建一列名为“示例列”，并随机赋予一些值
temp.insert(0, '序号', range(1, shapemax+ 1))
temp

,序号,日期,开始时间,持续时长/秒,日期,开始时间,持续时长/秒,日期,开始时间,持续时长/秒,...,持续时长/秒,日期,开始时间,持续时长/秒,日期,开始时间,持续时长/秒,日期,开始时间,持续时长/秒
0,1,4,12520,89,113,7076,7,23,19179,69,...,1,1,1604,1,1,10302,1,1,22725,2
1,2,8,2877,80,192,7024,1,23,19258,1,...,1,1,3103,1,1,21947,1,1,22865,2
2,3,11,2420,77,NaN,NaN,NaN,31,13465,98,...,1,1,3403,1,1,22467,1,1,22965,2
3,4,15,3156,93,NaN,NaN,NaN,31,13573,1,...,1,1,20184,1,1,25147,1,1,23145,2
4,5,26,1465,73,NaN,NaN,NaN,39,19102,71,...,1,1,22529,1,1,27227,1,1,23524,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8803,8804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,362,16057,4,NaN,NaN,NaN,NaN,NaN,NaN
8804,8805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,362,19639,1,NaN,NaN,NaN,NaN,NaN,NaN
8805,8806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,362,20279,1,NaN,NaN,NaN,NaN,NaN,NaN
8806,8807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,362,21939,1,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
temp.to_excel('result2_M202.xlsx',index=None)

149